<a href="https://colab.research.google.com/github/Martinaeht/Irony_Detection_roberta_bertweet/blob/master/Irony_Detection_MT_BERTweet_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Irony Detection - BERTweet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade
!pip install optuna
!pip install emoji==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 8.9 MB/s eta 0:00:00
  At

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, set_seed
from transformers import AutoModelForMaskedLM
from pprint import pprint
import numpy as np
import evaluate
import torch
from torch.utils.tensorboard import SummaryWriter
import os
import optuna
import json
import re
from sklearn.metrics import precision_score, recall_score, f1_score

set_seed(333) ## global seed

irony_dataset = load_dataset("cardiffnlp/tweet_eval", "irony")

train_dataset = irony_dataset['train']
validation_dataset = irony_dataset['validation']
test_dataset = irony_dataset['test']

train_dataset_print = train_dataset[:10]

for label, text in zip(train_dataset_print['label'], train_dataset_print['text']):
    print(label, text)

small_irony_dataset = DatasetDict(
    train = train_dataset.shuffle(seed=333).select(range(640)),
    val = validation_dataset.shuffle(seed=333).select(range(160)),
    test = test_dataset.shuffle(seed=333).select(range(160)))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2862 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/784 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/955 [00:00<?, ? examples/s]

1 seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life
0 look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵
1 Now I remember why I buy books online @user #servicewithasmile
1 @user @user So is he banded from wearing the clothes?  #Karma
1 Just found out there are Etch A Sketch apps.  #oldschool #notoldschool
1 Hey what do you know, one of the witnesses supporting Darren Wilson's story lied! And is racist! Mind blown!
0 @user on stage at #flzjingleball at the @user in #Tampa #iheartradio
1 You know it's going to be a great day when you're Garmin resets itself and you spill some cinnamon down yourself  #slowclap
1 Halfway thorough my workday ... Woooo
1 Would like to thank my nephew for giving me his horrible cold & sore throat etc.. Much appreciated!


In [ ]:
# debugging if needed

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

#def tokenize_function(examples):
#    return tokenizer(examples["text"], padding=True, truncation=True) # added truncation=True

max_length = 128  # Choose an appropriate maximum length for bertweet-base

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )

small_tokenized_dataset = small_irony_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)

accuracy = evaluate.load("accuracy")


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Check for invalid labels for debugging

invalid_labels = [label for label in small_tokenized_dataset['train']['label'] if label not in [0, 1]]
print(f"Invalid labels in the training dataset: {invalid_labels}")

invalid_labels_val = [label for label in small_tokenized_dataset['val']['label'] if label not in [0, 1]]
print(f"Invalid labels in the training dataset: {invalid_labels_val}")


Invalid labels in the training dataset: []
Invalid labels in the training dataset: []


# BERTweet manual finetuning

In [ ]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = (predictions == labels).mean()
  return {"accuracy": accuracy}

In [ ]:

arguments = TrainingArguments(
    output_dir="sample_cl_trainer_BERTweet",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs= 5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate= 2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=333
)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
# to test
print(small_tokenized_dataset['train'][2])

{'text': '@user @user |Yo boy! You guys are know to keep the party mood going. |#EzoneGoaDiaries.', 'label': 0, 'input_ids': [0, 5238, 4699, 5238, 4699, 61529, 2598, 7966, 12, 73, 296, 41, 68, 9, 264, 6, 548, 1462, 45006, 4, 61529, 1754, 685, 5485, 2216, 527, 1881, 9102, 783, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1,

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.678600,0.667873,0.556250
2,0.589600,0.626211,0.631250
3,0.464100,0.605920,0.662500
4,0.328200,0.662698,0.643750
5,0.305100,0.672829,0.656250


TrainOutput(global_step=200, training_loss=0.48808351635932923, metrics={'train_runtime': 5375.9045, 'train_samples_per_second': 0.595, 'train_steps_per_second': 0.037, 'total_flos': 210488844288000.0, 'train_loss': 0.48808351635932923, 'epoch': 5.0})

In [ ]:
save_directory = "/content/drive/MyDrive/Colab Notebooks/Project_Irony_Detection/BERTweet_pretrained_640"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [ ]:

output_dir_test = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_testing_man_finetuned_model_640"
if not os.path.exists(output_dir_test):
  os.makedirs(output_dir_test)

# adapted checkpoint
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/sample_cl_trainer_BERTweet/checkpoint-120", num_labels=2)

for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")

arguments_test_set = TrainingArguments(
    output_dir="results_test_set_bt",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer_test_set.evaluate(eval_dataset=small_tokenized_dataset['test'])
print(f"Test Accuracy: {test_results['eval_accuracy']}")

# output: Test Accuracy: 0.75625,
# compared to training accuracy: 0.662500 best epoch (checkpoint 120)


Text: #AdvancedWarfare is so much better than any FPS released in 2014 XD
Predicted Label: irony

Text: @user @user @user |*Everything is crossed*|#ChristmasGift |#BassFace
Predicted Label: non-irony

Text: Laborers in the San Joaquin Valley, one of the most productive farming regions in the world, suffer from food insecurity. #irony #poverty
Predicted Label: irony

Text: Pips who drop out of school n have Kirya as 'eir role model tht ey wana sing too..guy spix Portuguese,French,english...#Not a school dropout
Predicted Label: non-irony

Text: I really want to see Into The Woods!
Predicted Label: non-irony

Text: #Irony: al jazeera is pro Anti - #GamerGate because feminism, or something:
Predicted Label: irony

Text: Sleep. SLEEP YOU STUPID IDIOT I WANNA LEAVE MY ROOM
Predicted Label: non-irony

Text: Some days are just better than others..
Predicted Label: irony

Text: Christmas alone😊 how nice #not
Predicted Label: irony

Text: Read at the news '#Obama: #The #American #way #of #life 

Test Accuracy: 0.75625


In [ ]:
# retrofitted - F1 score

def compute_f1_score(model, tokenizer, dataset):
    all_labels = []
    all_predictions = []

    for example in dataset:
        text = example['text']
        label = example['label']
        model_inputs = tokenizer(text, return_tensors="pt")
        logits = model(**model_inputs).logits
        prediction = torch.argmax(logits, dim=-1).item()

        all_labels.append(label)
        all_predictions.append(prediction)

    precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

print(compute_f1_score(fine_tuned_model, tokenizer, small_tokenized_dataset['test']))


{'precision': 0.7112277041400753, 'recall': 0.7125, 'f1': 0.7117513020833333}


worst epoch

In [ ]:
# worst epoch

output_dir_test = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_testing_man_finetuned_model_640_worstcp40"
if not os.path.exists(output_dir_test):
  os.makedirs(output_dir_test)

# adapted checkpoint
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/sample_cl_trainer_BERTweet/checkpoint-40", num_labels=2)

for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")

arguments_test_set = TrainingArguments(
    output_dir="results_test_set_bt",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer_test_set.evaluate(eval_dataset=small_tokenized_dataset['test'])
print(f"Test Accuracy: {test_results['eval_accuracy']}")

# output: Test Accuracy: 0.6625,
# compared to training accuracy:  0.556250 worst epoch (checkpoint 40)


NameError: name 'os' is not defined

# BERTweet visualizing manually finetuned model

In [ ]:
# Tensors on different devices --> solution:

# Determine the device (CUDA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the correct device
fine_tuned_model.to(device)

# Tokenize the input text and move it to the same device
model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:

path = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_results_vis_640_manual_bestcp"
if not os.path.exists(path):
    os.mkdir(path)

layer = 0

while layer in range(len(outputs.hidden_states)):
    layer_path = os.path.join(path, f'layer_{layer}')
    if not os.path.exists(layer_path):
        os.mkdir(layer_path)

    example = 0
    tensors = []
    labels = []

    while example in range(len(outputs['hidden_states'][layer])):
      tensor = outputs['hidden_states'][layer][example][0]
      tensors.append(tensor)

      label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
      labels.append(label)
      example +=1

    if tensors:
        writer = SummaryWriter(layer_path)
        writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text', 'label'])
        writer.close()
        print(f"Embeddings for layer {layer} added to TensorBoard.")

    layer += 1


Embeddings for layer 0 added to TensorBoard.
Embeddings for layer 1 added to TensorBoard.
Embeddings for layer 2 added to TensorBoard.
Embeddings for layer 3 added to TensorBoard.
Embeddings for layer 4 added to TensorBoard.
Embeddings for layer 5 added to TensorBoard.
Embeddings for layer 6 added to TensorBoard.
Embeddings for layer 7 added to TensorBoard.
Embeddings for layer 8 added to TensorBoard.
Embeddings for layer 9 added to TensorBoard.
Embeddings for layer 10 added to TensorBoard.
Embeddings for layer 11 added to TensorBoard.
Embeddings for layer 12 added to TensorBoard.


worst epoch

In [ ]:
# worst epoch

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/sample_cl_trainer_BERTweet/checkpoint-40", num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fine_tuned_model.to(device)

model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:

path = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_results_vis_640_manual_worstcp40"
if not os.path.exists(path):
    os.mkdir(path)

layer = 0

while layer in range(len(outputs.hidden_states)):
    layer_path = os.path.join(path, f'layer_{layer}')
    if not os.path.exists(layer_path):
        os.mkdir(layer_path)

    example = 0
    tensors = []
    labels = []

    while example in range(len(outputs['hidden_states'][layer])):
      tensor = outputs['hidden_states'][layer][example][0] # [sp_token_position]
      tensors.append(tensor)

      label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
      labels.append(label)
      example +=1

    if tensors:
        writer = SummaryWriter(layer_path)
        writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text', 'label'])
        writer.close()
        print(f"Embeddings for layer {layer} added to TensorBoard.")

    layer += 1


Embeddings for layer 0 added to TensorBoard.
Embeddings for layer 1 added to TensorBoard.
Embeddings for layer 2 added to TensorBoard.
Embeddings for layer 3 added to TensorBoard.
Embeddings for layer 4 added to TensorBoard.
Embeddings for layer 5 added to TensorBoard.
Embeddings for layer 6 added to TensorBoard.
Embeddings for layer 7 added to TensorBoard.
Embeddings for layer 8 added to TensorBoard.
Embeddings for layer 9 added to TensorBoard.
Embeddings for layer 10 added to TensorBoard.
Embeddings for layer 11 added to TensorBoard.
Embeddings for layer 12 added to TensorBoard.


# BERTweet Optuna

In [ ]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = (predictions == labels).mean()
  return {"accuracy": accuracy}

In [ ]:
# Optuna objective function

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 6)
    weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1) # suggest_float

    arguments = TrainingArguments(
        output_dir="sample_cl_trainer_BERTweet_optuna_640_2",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_steps=8,
        num_train_epochs=num_train_epochs,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        report_to='none',
        seed=333,
        #lr_scheduler_type='linear',  # Added scheduler
        warmup_steps=500  # Added warmup steps to avoid early overfitting
    )

    trainer = Trainer(
        model=model,
        args=arguments,
        train_dataset=small_tokenized_dataset['train'],
        eval_dataset=small_tokenized_dataset['val'],
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()

    eval_results = trainer.evaluate()

    return eval_results["eval_accuracy"]




In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print(f"Best trial: {study.best_trial.params}")

[I 2025-01-28 10:58:59,514] A new study created in memory with name: no-name-3a3af6a9-4fa4-4053-a71b-5cf9e62b7456
<ipython-input-8-da64ed98416e>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
<ipython-input-8-da64ed98416e>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1) # suggest_float


Epoch,Training Loss,Validation Loss,Accuracy
1,0.691500,0.688902,0.562500
2,0.676700,0.682096,0.631250
3,0.663000,0.673134,0.593750


[I 2025-01-28 11:00:47,840] Trial 0 finished with value: 0.59375 and parameters: {'learning_rate': 3.143470880712429e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.06734160949316867}. Best is trial 0 with value: 0.59375.
<ipython-input-8-da64ed98416e>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
<ipython-input-8-da64ed98416e>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1) # suggest_float


Epoch,Training Loss,Validation Loss,Accuracy
1,0.605800,0.666144,0.543750
2,0.430200,0.588317,0.718750
3,0.410600,1.279975,0.512500
4,0.332500,1.089747,0.675000
5,0.265000,0.846038,0.700000


[I 2025-01-28 11:03:16,580] Trial 1 finished with value: 0.71875 and parameters: {'learning_rate': 0.0003057844063175081, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.006792285220825478}. Best is trial 1 with value: 0.71875.
<ipython-input-8-da64ed98416e>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
<ipython-input-8-da64ed98416e>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1) # suggest_float


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.136700,0.718315,0.718750
2,0.258200,0.844199,0.718750
3,0.186100,1.072009,0.712500
4,0.212700,1.352330,0.687500
5,0.102000,1.683656,0.675000


[I 2025-01-28 11:25:55,217] Trial 8 finished with value: 0.71875 and parameters: {'learning_rate': 1.9807246388083593e-05, 'batch_size': 8, 'num_train_epochs': 5, 'weight_decay': 0.0036100039655069583}. Best is trial 1 with value: 0.71875.
<ipython-input-8-da64ed98416e>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
<ipython-input-8-da64ed98416e>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.0, 0.1) # suggest_float


Epoch,Training Loss,Validation Loss,Accuracy
1,0.107400,0.734267,0.725000
2,0.151300,0.867222,0.700000
3,0.165000,0.777529,0.718750
4,0.120600,1.003709,0.712500


[I 2025-01-28 11:28:02,341] Trial 9 finished with value: 0.725 and parameters: {'learning_rate': 2.2890928268312175e-05, 'batch_size': 16, 'num_train_epochs': 4, 'weight_decay': 0.008773885897852397}. Best is trial 9 with value: 0.725.


Best trial: {'learning_rate': 2.2890928268312175e-05, 'batch_size': 16, 'num_train_epochs': 4, 'weight_decay': 0.008773885897852397}


In [ ]:
save_directory = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_optuna_640_2"
if not os.path.exists(save_directory):
  os.makedirs(save_directory)

best_params_path = f"{save_directory}/best_params.json"
with open(best_params_path, 'w') as f:
    json.dump(study.best_trial.params, f)

print(f"Best trial parameters saved to {best_params_path}")

tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)


Best trial parameters saved to /content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_optuna_640_2/best_params.json


In [ ]:

with open(best_params_path, 'r') as f:
    best_params = json.load(f)

print("Best trial parameters loaded: ", best_params)

model = AutoModelForSequenceClassification.from_pretrained(save_directory, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# Final Training with optimal hyperparameters
arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_final_model_trainer_640_2",
    per_device_train_batch_size=best_params['batch_size'], # 'per_device_train_batch_size'
    per_device_eval_batch_size=best_params['batch_size'], # 'per_device_eval_batch_size'
    logging_steps=8,
    num_train_epochs=best_params['num_train_epochs'],
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params['learning_rate'],
    weight_decay=best_params['weight_decay'],
    load_best_model_at_end=True,
    report_to='none',
    seed=333
)

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Best trial parameters loaded:  {'learning_rate': 2.2890928268312175e-05, 'batch_size': 16, 'num_train_epochs': 4, 'weight_decay': 0.008773885897852397}


Epoch,Training Loss,Validation Loss,Accuracy
1,0.093200,1.030086,0.662500
2,0.102100,1.137625,0.725000
3,0.056900,1.335695,0.700000
4,0.012100,1.250812,0.706250


TrainOutput(global_step=160, training_loss=0.07194383395835757, metrics={'train_runtime': 130.0315, 'train_samples_per_second': 19.688, 'train_steps_per_second': 1.23, 'total_flos': 168391075430400.0, 'train_loss': 0.07194383395835757, 'epoch': 4.0})

# BERTweet testing the Optuna model

In [ ]:

output_dir_test = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_testing_finetuned_model_640_2"
if not os.path.exists(output_dir_test):
  os.makedirs(output_dir_test)

# adapted checkpoint
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_final_model_trainer_640_2/checkpoint-160", num_labels=2)

for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model(**model_inputs).logits, dim=-1)
  prediction = torch.argmax(fine_tuned_model(**model_inputs).logits) # added
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")


arguments_test_set = TrainingArguments(
    output_dir="results_test_set_opt_640_2",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    #processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer_test_set.evaluate()
print(f"Test Accuracy: {test_results['eval_accuracy']}")

# output: Test Accuracy: 0.79375,
# compared to training accuracy: 0.725000 best ep 2(cp160)


NameError: name 'os' is not defined

In [ ]:
# F1 score

def compute_f1_score(model, tokenizer, dataset, device='cuda'):
    model.to(device)
    all_labels = []
    all_predictions = []

    for example in dataset:
        text = example['text']
        label = example['label']
        model_inputs = tokenizer(text, return_tensors="pt").to(device)
        logits = model(**model_inputs).logits
        prediction = torch.argmax(logits, dim=-1).item()

        all_labels.append(label)
        all_predictions.append(prediction)

    precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

print(compute_f1_score(fine_tuned_model, tokenizer, small_tokenized_dataset['test'], device='cuda'))


{'precision': 0.8195684523809523, 'recall': 0.79375, 'f1': 0.7954418344519014}


worst epoch


In [ ]:
# worst epoch

output_dir_test_worstep = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_testing_finetuned_model_640_2_worstep_cp40"
if not os.path.exists(output_dir_test_worstep):
  os.makedirs(output_dir_test_worstep)

# adapted checkpoint
fine_tuned_model_worstep = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_final_model_trainer_640_2/checkpoint-40", num_labels=2)

for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model_worstep(**model_inputs).logits)
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")


arguments_test_set = TrainingArguments(
    output_dir="results_test_set_640_worstep",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model_worstep,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results_worstep = trainer_test_set.evaluate(eval_dataset=small_tokenized_dataset['test'])
print(f"Test Accuracy: {test_results_worstep['eval_accuracy']}")


#output: Test Accuracy: 0.73125
#compared to training accuracy:  0.662500 (cp40)


Text: #AdvancedWarfare is so much better than any FPS released in 2014 XD
Predicted Label: irony

Text: @user @user @user |*Everything is crossed*|#ChristmasGift |#BassFace
Predicted Label: irony

Text: Laborers in the San Joaquin Valley, one of the most productive farming regions in the world, suffer from food insecurity. #irony #poverty
Predicted Label: irony

Text: Pips who drop out of school n have Kirya as 'eir role model tht ey wana sing too..guy spix Portuguese,French,english...#Not a school dropout
Predicted Label: irony

Text: I really want to see Into The Woods!
Predicted Label: non-irony

Text: #Irony: al jazeera is pro Anti - #GamerGate because feminism, or something:
Predicted Label: irony

Text: Sleep. SLEEP YOU STUPID IDIOT I WANNA LEAVE MY ROOM
Predicted Label: irony

Text: Some days are just better than others..
Predicted Label: non-irony

Text: Christmas alone😊 how nice #not
Predicted Label: irony

Text: Read at the news '#Obama: #The #American #way #of #life #is #not

Test Accuracy: 0.73125


# BERTweet visualizing Optuna model

In [ ]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_final_model_trainer_640_2/checkpoint-160", num_labels=2)

# Tensors on different devices --> solution:
# Determine the device (CUDA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the correct device
fine_tuned_model.to(device)

# Tokenize the input text and move it to the same device
model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:

path = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_results_vis_opt_640_2_bestep_cp160_new"
if not os.path.exists(path):
    os.mkdir(path)

layer = 0

while layer in range(len(outputs.hidden_states)):
    layer_path = os.path.join(path, f'layer_{layer}')
    if not os.path.exists(layer_path):
        os.mkdir(layer_path)

    example = 0
    tensors = []
    labels = []

    while example in range(len(outputs['hidden_states'][layer])):
      tensor = outputs['hidden_states'][layer][example][0]
      tensors.append(tensor)

      label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
      labels.append(label)
      example +=1

    if tensors:
        writer = SummaryWriter(layer_path)
        writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text', 'label'])
        writer.close()
        print(f"Embeddings for layer {layer} added to TensorBoard.")

    layer += 1


Embeddings for layer 0 added to TensorBoard.
Embeddings for layer 1 added to TensorBoard.
Embeddings for layer 2 added to TensorBoard.
Embeddings for layer 3 added to TensorBoard.
Embeddings for layer 4 added to TensorBoard.
Embeddings for layer 5 added to TensorBoard.
Embeddings for layer 6 added to TensorBoard.
Embeddings for layer 7 added to TensorBoard.
Embeddings for layer 8 added to TensorBoard.
Embeddings for layer 9 added to TensorBoard.
Embeddings for layer 10 added to TensorBoard.
Embeddings for layer 11 added to TensorBoard.
Embeddings for layer 12 added to TensorBoard.


worst epoch

In [ ]:
fine_tuned_model_worstep = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_final_model_trainer_640_2/checkpoint-40", num_labels=2)

fine_tuned_model_worstep.to(device)

model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model_worstep(**model_inputs, output_hidden_states=True)


In [ ]:
# worst epoch

path_worstep = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/BERTweet_results_vis_opt_640_2_worstep_cp40_new"
if not os.path.exists(path_worstep):
  os.mkdir(path_worstep)

layer=0

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path_worstep+'/layer_' + str(layer)):
    os.mkdir(path_worstep+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
        tensor = outputs['hidden_states'][layer][example][0]
        tensors.append(tensor)
        label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
        labels.append(label)
        example +=1

  writer=SummaryWriter(path_worstep+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text','label'])

  layer+=1